In [13]:
# Imports
import json
import os
import csv

In [14]:
DATA_DIR = "/Users/user/Documents/Clades/July_2021"
label = "C.37"

ANNOTATION = os.path.join(DATA_DIR, label+"_annotation.json")
SUMMARY = os.path.join(DATA_DIR, label + "_summary.json")

In [15]:
def map_to_gene(site):
    global SUMMARY
    with open(SUMMARY) as f:
        data = json.load(f)
    #end with
    key = ""
    for key in data:
        #print(key)
        theMap = data[key]["map"]
        #rint(data[key]["map"])
        #print(type(theMap))
        #print(theMap)
        if int(site) in theMap:
            #print("Returning", key)
            return key, theMap.index(int(site))
        #end if
    return key, 0
#end method

In [16]:
NameMap = {"S": "s2Spike", "E": "s2Envelope", "N": "s2Nucleocapsid", "ORF3a": "s2ORF3a", "methyltransferase": "s2Methyltransferase",
           "helicase": "s2Helicase", "leader": "s2leader", "nsp10": "s2nsp10",
          "nsp2": "s2nsp2", "nsp3": "s2nsp3", "nsp4": "s2nsp4", "nsp6": "s2nsp6", 
           "nsp7": "s2nsp7", "nsp8": "s2nsp8", "nsp9": "s2nsp9",
          "ORF8": "s2ORF8", "ORF6": "s2ORF6", "3C": "s23C", "ORF7a": "s2ORF7a",
          "exonuclease": "s2Exonuclease", "M": "s2Matrix", "endornase":"s2Endornase", "RdRp": "s2RdRP"}

## Version 1

## The whole table.

In [17]:
# https://observablehq.com/@aglucaci/visualizing-selection-analysis-results-for-evolution-of-t/8

# Would be useful to have Table 1 results for spike
# also Table 2 (cfel)
# also the clade defining mutations

# return:
#.   genomic coordinate
#.   Codon in gene
#.   Num. selected branches? For meme not cfel
#.   p-value for meme, cfel
#.   q-value for meme, cfel sites
#.   Domain for site, use ncbi annotation
#.   AA frequency in clade
#.   AA frequency in reference

In [18]:
Gene = "N"

In [19]:
# Output to csv
def output_csv(data):
    global label, Gene
    with open(label + "_" + Gene +".csv", 'w') as csvfile:
        #pamwriter = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter = csv.writer(csvfile)

        for row in data:
            spamwriter.writerow(row)
            #spamwriter.writerow(['Spam'] * 5 + ['Baked Beans'])
            #spamwriter.writerow(['Spam', 'Lovely Spam', 'Wonderful Spam'])
        #end for
    #end with
#end method

def AA_frequencies(AA_dict):
    total = 0
    results = {}
    for item in AA_dict:
        total += AA_dict.get(item, 0)
    
    for item in AA_dict:
        AA = item
        freq = AA_dict.get(item, 0) / total
        results[AA] = round(freq,4)
        
    return results

In [20]:
# Domain annotation
# https://www.ncbi.nlm.nih.gov/nuccore/NC_045512
# https://www.ncbi.nlm.nih.gov/protein/1796318598

SPIKE_DOMAINS = {"NTD": [13, 304], "RBD": [319, 541], "Furin": [685, 656], "HR1": [918, 983], "HR2":[1162, 1203]}
# there are additional domains to add in https://www.ncbi.nlm.nih.gov/protein/1796318598
# also "Sites" like glycosylation sites.
CD = {"B-1-617-1": {"s2Spike": [452, 484, 614, 681, 1071],
                   "s2ORF3a": [26], "s2ORF7a":[82], "s2Nucleocapsid": [203, 377], "s2nsp2":[342, 397],
                   "s2RdRP":[192, 538, 857]}, 
      
      "B-1-617-2":{ "s2RdRP":[314, 662, 1000], 
                   "s2Spike": [19, 157,158, 452,478,614,681,950],
                   "s2ORF3a": [26],
                   "s2Matrix": [82],
                  "s2ORF7a": [82, 120],
                  "s2ORF8": [119, 120],
                  "s2Nucleocapsid": [63, 203, 377],},
      "C.37": {"s2Spike": [75, 76,452,490,614,859], "s2Nucleocapsid": [13, 203, 204, 214]}}

In [21]:
def process_sites(sl):
    global output_data, data, Gene
    #for site in sl: # site-list
    print("# Processing:", sl)
    for key in data:
        if data[key]["G"] != Gene:
            continue
        #end if
        #print(key, int(data[key]["S"]))
        #print(data[key][Gene])
        if int(data[key]["S"]) in sl: #site-list, S here stands for Site, not Spike
            #process it
            #print("found site")
            pval = data[key].get("bMEME", 1)
            if pval != 1: pval = data[key]["bMEME"]["p"]
            if pval <= 1.0:
                x, i = map_to_gene(key)
                branches_selected = str(data[key]["bMEME"]["br"])
                qval = str(-1)
                genomic_pos = key
                reference_tag = "Reference"
                AA_freq_clade = data[genomic_pos]["aa"][label]
                AA_freq_reference = data[genomic_pos]["aa"][reference_tag]
                AA_freq_clade = AA_frequencies(AA_freq_clade)
                AA_freq_reference = AA_frequencies(AA_freq_reference)
                h = AA_freq_clade
                AA_freq_clade = ""
                for item in h.keys():
                    AA_freq_clade += item + ":" + str(h[item]) + " / "
                j = AA_freq_reference
                AA_freq_reference = ""
                for item in j.keys():
                    AA_freq_reference += item + ":" + str(j[item]) + " / "
                # Domain
                domain = "S1"
                site = int(i) + 1
                for region in SPIKE_DOMAINS.keys():
                    start = SPIKE_DOMAINS[region][0]
                    end = SPIKE_DOMAINS[region][1]
                    if start <= site <= end:
                        domain = region
                        break
                    #end if
                if site >= 543: 
                    domain = "S2"  
                output = [key, str(site), domain, branches_selected, str(round(pval, 4)), qval, AA_freq_clade, AA_freq_reference]
                #print(",".join(output))
                print("# adding", output)
                output_data.append(output)
            #end if
        #end if
    #end for
#end method

In [22]:
# Get MEME Sites  -- Internal branches -- Table 1
output_data = []

# header
header = ["GenomicPosition", "Site", "Domain", "Num.BranchesSelected", "p-value", "q-value", "AA Freq.Clade", "AA Freq.Reference", "CladeKeys", "RefKeys"]
print(header)
output_data.append(header)

# initialize data
with open(ANNOTATION) as f:
    data = json.load(f)
#end with

# Do Clade defining sites
if Gene == "N":
    print("# Checking sites:", CD[label]["s2Nucleocapsid"])
    process_sites(CD[label]["s2Nucleocapsid"]) 
else:
    print("# Checking sites:", CD[label]["s2Spike"])
    process_sites(CD[label]["s2Spike"])
#end if

#cd_sl = process_sites(CD[label]["s2Spike"])
#for item in cd_sl: output_data.append(item)

for key in data: # keys are the sites, genomic position
    pval = 0
    pval = data[key].get("bMEME", 1)
    if pval != 1: 
        pval = data[key]["bMEME"]["p"]
    #end if
    if pval <= 0.05:
        x, i = map_to_gene(key)
        if x != Gene: 
            continue # only want spike or nucleocapsid
        #end if
        branches_selected = str(data[key]["bMEME"]["br"])
        qval = str(-1)
        #domain = "N/A"
        
        genomic_pos = key
        reference_tag = "Reference"
        AA_freq_clade = data[genomic_pos]["aa"][label]
        AA_freq_reference = data[genomic_pos]["aa"][reference_tag]
        AA_freq_clade = AA_frequencies(AA_freq_clade)
        AA_freq_reference = AA_frequencies(AA_freq_reference)
        h = AA_freq_clade
        AA_freq_clade = ""
        for item in h.keys():
            AA_freq_clade += item + ":" + str(h[item]) + " / "
            
        j = AA_freq_reference
        AA_freq_reference = ""
        for item in j.keys():
            AA_freq_reference += item + ":" + str(j[item]) + " / "
            
        # Domain
        domain = "S1"
        site = int(i)+1
        
        for region in SPIKE_DOMAINS.keys():
            start = SPIKE_DOMAINS[region][0]
            end = SPIKE_DOMAINS[region][1]
            if start <= site <= end:
                domain = region
                break
        
        if site >= 543: domain = "S2"  
        output = [key, str(site), domain, branches_selected, str(round(pval, 4)), 
                  qval, AA_freq_clade, AA_freq_reference, 
                  str(h.keys()), str(j.keys())]
        print(",".join(output))
        output_data.append(output)
    #end if
#end for
        
# Do cfel here.
cfel_sites = []

for key in data:
    pval = data[key]["bCFEL"]["pp"]
    if abs(pval) <= 0.1:
        x, i = map_to_gene(key)
        if x != "S": continue
        #print(x, NameMap[x], int(i)+1, int(i)+1, 0.599)
        cfel_sites.append (int(i)+1)
process_sites(cfel_sites)

print("# writing")     
output_csv(output_data)
print("# done.")

['GenomicPosition', 'Site', 'Domain', 'Num.BranchesSelected', 'p-value', 'q-value', 'AA Freq.Clade', 'AA Freq.Reference', 'CladeKeys', 'RefKeys']
# Checking sites: [13, 203, 204, 214]
# Processing: [13, 203, 204, 214]
# adding ['28309', '13', 'NTD', '2', '0.0889', '-1', 'P:0.0588 / L:0.916 / F:0.0126 / I:0.0042 / -:0.0084 / ', 'P:0.952 / L:0.024 / S:0.008 / T:0.008 / H:0.008 / ']
# adding ['28879', '203', 'NTD', '0', '0.6667', '-1', 'K:0.9832 / -:0.0084 / R:0.0084 / ', 'R:0.744 / K:0.24 / M:0.008 / G:0.008 / ']
# adding ['28882', '204', 'NTD', '0', '0.6667', '-1', 'R:0.9832 / -:0.0084 / G:0.0084 / ', 'G:0.768 / T:0.008 / L:0.008 / R:0.2 / -:0.016 / ']
# adding ['28912', '214', 'NTD', '0', '0.2152', '-1', 'G:0.0462 / S:0.0042 / C:0.937 / -:0.0126 / ', 'G:0.992 / C:0.008 / ']
28303,11,S1,1,0.0458,-1,N:0.9832 / T:0.0042 / -:0.0042 / D:0.0084 / ,N:0.992 / S:0.008 / ,dict_keys(['N', 'T', '-', 'D']),dict_keys(['N', 'S'])
28324,18,NTD,1,0.0378,-1,G:0.9874 / V:0.0084 / -:0.0042 / ,G:0.976 / V:

In [23]:
import pandas as pd
df = pd.read_csv(label + "_" + Gene +".csv")
#df.index += 1
df = df.drop(columns=['q-value', 'CladeKeys', 'RefKeys'])
sorted_df = df.sort_values(by='Site', ascending=True)
#df = df.rename_axis('MyIdx').sort_values(by = ['MyCol', 'MyIdx'], ascending = [False, True])
sorted_df = sorted_df.reset_index(drop=True)
sorted_df.index += 1
print(sorted_df.to_markdown())

|    |   GenomicPosition |   Site | Domain   |   Num.BranchesSelected |   p-value | AA Freq.Clade                                          | AA Freq.Reference                                 |
|---:|------------------:|-------:|:---------|-----------------------:|----------:|:-------------------------------------------------------|:--------------------------------------------------|
|  1 |             28303 |     11 | S1       |                      1 |    0.0458 | N:0.9832 / T:0.0042 / -:0.0042 / D:0.0084 /            | N:0.992 / S:0.008 /                               |
|  2 |             28309 |     13 | NTD      |                      2 |    0.0889 | P:0.0588 / L:0.916 / F:0.0126 / I:0.0042 / -:0.0084 /  | P:0.952 / L:0.024 / S:0.008 / T:0.008 / H:0.008 / |
|  3 |             28324 |     18 | NTD      |                      1 |    0.0378 | G:0.9874 / V:0.0084 / -:0.0042 /                       | G:0.976 / V:0.024 /                               |
|  4 |             28342 |     24 |